# mobile V2

In [10]:
from tensorflow.keras import layers, Model, Sequential
# from model import MobileNetV2
import tensorflow as tf
import numpy as np
import os
import time
import glob
import random

In [11]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"



gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        exit(-1)

In [4]:
def _make_divisible(ch, divisor=8, min_ch=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    """
    if min_ch is None:
        min_ch = divisor
    new_ch = max(min_ch, int(ch + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_ch < 0.9 * ch:
        new_ch += divisor
    return new_ch

In [5]:
class ConvBNReLU(layers.Layer):
    def __init__(self, out_channel, kernel_size=3, stride=1, **kwargs):
        super(ConvBNReLU, self).__init__(**kwargs)
        self.conv = layers.Conv2D(filters=out_channel, kernel_size=kernel_size,
                                  strides=stride, padding='SAME', use_bias=False, name='Conv2d')  # 迁移学习中调用权重信息用到 name
        self.bn = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name='BatchNorm')
        # ReLU6
        self.activation = layers.ReLU(max_value=6.0)

    def call(self, inputs, training=False, **kwargs):
        x = self.conv(inputs)
        x = self.bn(x, training=training)
        x = self.activation(x)
        return x

In [6]:
class InvertedResidual(layers.Layer):
    # expand_ratio 控制特征矩阵的深度
    def __init__(self, in_channel, out_channel, stride, expand_ratio, **kwargs):
        super(InvertedResidual, self).__init__(**kwargs)
        self.hidden_channel = in_channel * expand_ratio
        self.use_shortcut = stride == 1 and in_channel == out_channel

        layer_list = []
        if expand_ratio != 1:
            # 1x1 pointwise conv
            layer_list.append(ConvBNReLU(out_channel=self.hidden_channel, kernel_size=1, name='expand'))

        ## extend TODO
        layer_list.extend([
            # 3x3 depthwise conv
            layers.DepthwiseConv2D(kernel_size=3, padding='SAME', strides=stride,
                                   use_bias=False, name='depthwise'),
            layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name='depthwise/BatchNorm'),
            layers.ReLU(max_value=6.0),
            # 1x1 pointwise conv(linear)
            layers.Conv2D(filters=out_channel, kernel_size=1, strides=1,
                          padding='SAME', use_bias=False, name='project'),
            layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name='project/BatchNorm')
        ])
        # 主分支的输出
        self.main_branch = Sequential(layer_list, name='expanded_conv')

    def call(self, inputs, **kwargs):
        if self.use_shortcut:
            return inputs + self.main_branch(inputs)
        else:
            return self.main_branch(inputs)

In [29]:
# 模型框架
# alpha=1.0 调节卷积核个数，round_nearest 调整卷积个数为 8 的倍数
def MobileNetV2(im_height=224, im_width=224, im_channel=3, num_classes=1000, alpha=1.0, round_nearest=8):
    block = InvertedResidual
    input_channel = _make_divisible(32 * alpha, round_nearest)
    last_channel = _make_divisible(1280 * alpha, round_nearest)
    
    # t 扩展因子， c 输入特征的深度， n bottleneck 的重复次数
    inverted_residual_setting = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]

    input_image = layers.Input(shape=(im_height, im_width, im_channel), dtype='float32')
    # conv1
    x = ConvBNReLU(input_channel, stride=2, name='Conv')(input_image)   # 此处默认卷积核大小为 3
    # building inverted residual residual blockes
    for t, c, n, s in inverted_residual_setting:
        output_channel = _make_divisible(c * alpha, round_nearest)
        for i in range(n):
            stride = s if i == 0 else 1
            x = block(x.shape[-1], output_channel, stride, expand_ratio=t)(x)
    # building last several layers
    x = ConvBNReLU(last_channel, kernel_size=1, name='Conv_1')(x)

    # building classifier
    x = layers.GlobalAveragePooling2D()(x)  # pool + flatten
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(num_classes, name='Logits')(x)

    model = Model(inputs=input_image, outputs=output)
    return model

In [30]:
data_dir= "E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D"
data_root = glob.glob(data_dir + '/*')
for name in glob.glob(data_dir + '/*'):
    print(name)

E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\data.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\data_label.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\test.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\test_label.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\train.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\train_label.npy


In [31]:
train = np.load(data_root[4])
train_label = np.load(data_root[5])
test = np.load(data_root[2])
test_label = np.load(data_root[3])
train.shape, train_label.shape, test.shape, test_label.shape

((3200, 9, 9, 204), (3200,), (50929, 9, 9, 204), (50929,))

In [32]:
BATCHSZ = 64
batch_size = 64
class_num = 16
im_height = 9
im_width = 9
epochs = 10
im_channel = train.shape[3]
train_num = train.shape[0]
val_num = test.shape[0]

In [33]:
# 根据可用的CPU动态设置并行调用的数量， 应用于 num_parallel_calls

train_label = tf.keras.utils.to_categorical(train_label)
test_label = tf.keras.utils.to_categorical(test_label)

# 根据可用的CPU动态设置并行调用的数量， 应用于 num_parallel_calls
AUTOTUNE = tf.data.experimental.AUTOTUNE

# load train dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train, train_label))
train_dataset = train_dataset.shuffle(buffer_size=train_num).repeat().batch(BATCHSZ).prefetch(AUTOTUNE)
# train_dataset = train_dataset.shuffle(buffer_size=train_num).batch(BATCHSZ).prefetch(AUTOTUNE)

# load test dataset
val_dataset = tf.data.Dataset.from_tensor_slices((test, test_label))
val_dataset = val_dataset.repeat().batch(BATCHSZ).prefetch(AUTOTUNE)
# val_dataset = val_dataset.batch(BATCHSZ).prefetch(AUTOTUNE)

In [34]:
train_dataset, val_dataset 

(<PrefetchDataset shapes: ((None, 9, 9, 204), (None, 16)), types: (tf.float32, tf.float32)>,
 <PrefetchDataset shapes: ((None, 9, 9, 204), (None, 16)), types: (tf.float32, tf.float32)>)

In [35]:
# create direction for saving weights
if not os.path.exists("save_weights"):
    os.makedirs("save_weights")


# 实例化模型
model = MobileNetV2(im_height=im_height, im_width=im_width, im_channel=im_channel, num_classes=class_num, alpha=1.0, round_nearest=8)
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 9, 9, 204)]       0         
_________________________________________________________________
Conv (ConvBNReLU)            (None, 5, 5, 32)          58880     
_________________________________________________________________
inverted_residual_51 (Invert (None, 5, 5, 16)          992       
_________________________________________________________________
inverted_residual_52 (Invert (None, 3, 3, 24)          5568      
_________________________________________________________________
inverted_residual_53 (Invert (None, 3, 3, 24)          9456      
_________________________________________________________________
inverted_residual_54 (Invert (None, 2, 2, 32)          10640     
_________________________________________________________________
inverted_residual_55 (Invert (None, 2, 2, 32)         

In [36]:
# using keras low level api for training
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [37]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        output = model(images, training=True)
        loss = loss_object(labels, output)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, output)

In [38]:
@tf.function
def test_step(images, labels):
    output = model(images, training=False)
    t_loss = loss_object(labels, output)

    test_loss(t_loss)
    test_accuracy(labels, output)

In [41]:
epochs = 100

In [40]:
best_test_loss = float('inf')
train_step_num = train_num // batch_size
val_step_num = val_num // batch_size
for epoch in range(1, epochs+1):
    train_loss.reset_states()        # clear history info
    train_accuracy.reset_states()    # clear history info
    test_loss.reset_states()         # clear history info
    test_accuracy.reset_states()     # clear history info

    t1 = time.perf_counter()
    for index, (images, labels) in enumerate(train_dataset):
        train_step(images, labels)
        if index+1 == train_step_num:
            break
    print(time.perf_counter()-t1)

    for index, (images, labels) in enumerate(val_dataset):
        test_step(images, labels)
        if index+1 == val_step_num:
            break

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch,
                            train_loss.result(),
                            train_accuracy.result() * 100,
                            test_loss.result(),
                            test_accuracy.result() * 100))
    if test_loss.result() < best_test_loss:
        model.save_weights("./save_weights/myMobileNet.ckpt".format(epoch), save_format='tf')

5.68980380000005
Epoch 1, Loss: 2.066819190979004, Accuracy: 34.34375, Test Loss: 3.0094127655029297, Test Accuracy: 3.4905660152435303
0.677453200000059
Epoch 2, Loss: 1.0890570878982544, Accuracy: 62.343753814697266, Test Loss: 1.440706729888916, Test Accuracy: 48.724449157714844
0.679537200000027
Epoch 3, Loss: 0.74652099609375, Accuracy: 73.5625, Test Loss: 0.9158334136009216, Test Accuracy: 65.71934509277344
0.6854324999999335
Epoch 4, Loss: 0.6209841370582581, Accuracy: 77.96875, Test Loss: 0.5073484182357788, Test Accuracy: 74.6128158569336
0.7016564000000471
Epoch 5, Loss: 0.4849494993686676, Accuracy: 82.78125, Test Loss: 0.43151625990867615, Test Accuracy: 82.22877502441406
0.7017526999999291
Epoch 6, Loss: 0.5203126072883606, Accuracy: 82.21875, Test Loss: 0.7346500754356384, Test Accuracy: 65.04913330078125
0.6898727000000235
Epoch 7, Loss: 0.40045440196990967, Accuracy: 85.0625, Test Loss: 0.4427463114261627, Test Accuracy: 82.78498840332031
0.7090054000000237
Epoch 8, Los